In [1]:
import ast
import inspect

In [2]:
def path_of_file(lib_path=None):
    if lib_path is not None:
        import sys
        import os
        sys.path.insert(0,lib_path)
        with open(F"{lib_path}", "r") as f:
            code = f.read() #
        tree = ast.parse(code)
        return tree

In [3]:
class WildcardImportChecker(ast.NodeVisitor):
    def visit_ImportFrom(self, node):
        for alias in node.names:
            self._file_module[f"{alias.name}"]=f'{node.module}'
        self.generic_visit(node)
    def set_file_module(self, module):
        self._file_module = module
        
def import_module(tree):
#     tree = path_of_file(file_path)
    file_module={}
    visitor = WildcardImportChecker()
    visitor.set_file_module(file_module)
    visitor.visit(tree)
    return file_module
    
# h=r"C:/Users/Dell/Desktop/storage/timeline/2023/july/auto access relate code/input.py"    
# print(import_module(h))

In [37]:
class PrintToCallFunctionTransformer(ast.NodeTransformer):
    def visit_Call(self, node):
        if node.func.id!='print':
            self._function+=[f'{node.func.id}']
        self.generic_visit(node)
    def set_function(self,id):
        self._function = id

def import_function_name(tree):
#     tree = path_of_file(file_path)
    function=[]
    transformer = PrintToCallFunctionTransformer()
    transformer.set_function(function)
    transformer.visit(tree)
    return function

# h=r"C:/Users/Dell/Desktop/storage/timeline/2023/july/auto access relate code/input.py"    
# print(import_function_name())

NameError: name 'tree' is not defined

In [52]:
def find_assign_nodes(source_code):
    tree = ast.parse(source_code)
    assign_nodes = [node for node in ast.walk(tree) if isinstance(node, ast.Assign)]
    return assign_nodes

In [ ]:
def exclude_imports_functions(code_str):
    parsed_ast = ast.parse(code_str)
    def reconstruct_code(node):
        if isinstance(node, (ast.Import, ast.ImportFrom)):
            return ""
        return ast.unparse(node)
    filtered_code = "\n".join(reconstruct_code(node) for node in parsed_ast.body)
    return filtered_code

In [56]:
def call_function(function):
    x_list=set()
    assign_nodes = find_assign_nodes(function)
    for assign_node in assign_nodes:
        targets = [target.id for target in assign_node.targets if isinstance(target, ast.Name)]
        value = ast.unparse(assign_node.value).strip()
        x=''
        if len(targets)==0:
            for j in value:
                if j=='(':
                    x_list.add(x)
                    break
                else:
                    x+=j
    return x_list

# call_function(function)

In [6]:
def fileOpen(outputFile, content):
    with open(outputFile,'a') as f:
        f.write(content)
        f.close()

In [50]:
def find_callFunction_inside_function():
    source = (open("output.py", "r").read())
    functions = [f.name for f in ast.parse(source).body
         if isinstance(f, ast.FunctionDef)]
    return functions

find_callFunction_inside_function()

['bidderAndBidReadOnly', 'main']

In [59]:
def newFile():
    tree=path_of_file('input.py')
    import_file=import_module(tree)
    function=import_function_name(tree)
#     print(function)
    for i in function:
        module = __import__(str(import_file[i]))#file name 
        source_code = inspect.getsource(getattr(module,i))#def func name
        inside_function=call_function(source_code)
        
        print(inside_function,'hello')
        
#     print(functions)
#         if len(functions)!=0:
#             print(function,functions)
#             remain_function_write=set(function).difference(set(functions))
#             print(remain_function_write)
#         for j in remain_function_write:
#             print(j)
#             source_code = inspect.getsource(getattr(module,j))#def func name
#             fileOpen(outputFile,source_code)

        
#     filtered_code = exclude_imports_functions(code)
#     file=fileOpen()
#     file.write(filtered_code)
#     file.close()

newFile() 

set() hello
{'bidderAndBid', 'bidderAndBidReadOnly'} hello
